In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
from tqdm import tqdm

In [3]:
import malaya

In [4]:
# model = malaya.dependency.transformer(model = 'xlnet', quantized = True)

In [6]:
# https://ms.wikipedia.org/wiki/Penjodoh_bilangan_bahasa_Melayu
penjodoh_bilangan = [
    'angkatan',
    'baris',
    'batang',
    'bentuk',
    'bidang',
    'biji',
    'bilah',
    'buah',
    'buku',
    'bungkus',
    'butir',
    'carik',
    'cebis',
    'cekak',
    'cubit',
    'cucuk',
    'das',
    'deret',
    'ekor',
    'gugus',
    'gelung',
    'gemal',
    'genggam',
    'gulung',
    'gumpal',
    'helai',
    'hidangan',
    'hiris',
    'ikat',
    'jambak',
    'jambangan',
    'jemput',
    'kaki',
    'kalung',
    'kandang',
    'kapur',
    'kawan',
    'kelompok',
    'kepal',
    'keping',
    'kepul',
    'kerat',
    'ketul',
    'kotak',
    'kuntum',
    'laras',
    'lembar',
    'lingkar',
    'longgok',
    'naskhah',
    'orang',
    'papan',
    'pasang',
    'pasukan',
    'patah',
    'pintu',
    'potong',
    'pucuk',
    'puntung',
    'rangkap',
    'rawan',
    'ruas',
    'rumpun',
    'sikat',
    'sisir',
    'suap',
    'tandan',
    'tangkai',
    'teguk',
    'timbun',
    'titik',
    'tongkol',
    'ulas',
    'untai',
    'urat',
    'utas',
]
hubung_list = [
    'agar',
    'apabila',
    'atau',
    'bahawa',
    'dan',
    'hingga',
    'jika',
    'jikalau',
    'kecuali',
    'kerana',
    'lalu',
    'manakala',
    'sambil',
    'serta',
    'semenjak',
    'sementara',
    'sungguhpun',
    'supaya',
    'walaupun',
    'tetapi',
    'berkenan',
    'berkenaan',
    'yang',
    'juga',
    'tersebut'
]

In [7]:
sepenjodoh_bilangan = [f'se{w}' for w in penjodoh_bilangan]
set_sepenjodoh_bilangan = set(sepenjodoh_bilangan)
set_penjodoh_bilangan = set(penjodoh_bilangan)

In [8]:
with open('filtered-dumping-wiki.txt') as fopen:
    data = list(filter(None, fopen.read().split('\n')))
    
data = [i for i in data if len(i) >= 2]

len(data)

2037249

In [9]:
tokenizer = malaya.preprocessing.TOKENIZER().tokenize

In [10]:
# graph, tagging, indexing = model.predict(' '.join(tokenizer('ibu bapa kita bekerja penat lelah')))
# graph.to_graphvis()

In [12]:
import copy
import re
from malaya.text.regex import _expressions
import random

def reset_t(tokens):
    t = []
    for i in range(len(tokens)):
        t.append([tokens[i], 2])
    return t

# [penjodoh bilangan] [kata nama] -> [wrong penjodoh bilangan] [kata nama - kata nama]
# dua buah kereta -> dua biji kereta
def augment_10_0(t, row):
    text, tokens, tokens_lower, penjodoh = row
    for word in penjodoh:
        try:
            i = tokens_lower.index(word)
            negate = list(set_penjodoh_bilangan - {word})
            choice = random.choice(negate)
            t[i][0] = choice
            t[i][1] = 10
        except Exception as e:
            print(e)
            pass

# se[penjodoh bilangan] [kata nama] -> se[wrong penjodoh bilangan] [kata nama - kata nama]
# sebuah kereta -> sebiji kereta
def augment_10_1(t, row):
    text, tokens, tokens_lower, penjodoh = row
    for word in penjodoh:
        try:
            i = tokens_lower.index(word)
            negate = list(set_sepenjodoh_bilangan - {word})
            choice = random.choice(negate)
            t[i][0] = choice
            t[i][1] = 10
        except Exception as e:
            print(e)
            pass

In [15]:
results = []
for text in tqdm(['2 buah kereta']):
    tokens = tokenizer(text)
    t = reset_t(tokens)
    t_ = copy.deepcopy(t)
    tokens_lower = tokenizer(text.lower())
    set_tokens = set(tokens_lower)
    r_penjodoh_bilangan = set_tokens & set_penjodoh_bilangan
    r_sepenjodoh_bilangan = set_tokens & set_sepenjodoh_bilangan
    r = (t, tokens, tokens_lower, r_penjodoh_bilangan)
    augment_10_0(t_, r)
    r = (t, tokens, tokens_lower, r_sepenjodoh_bilangan)
    augment_10_1(t_, r)
    results.append((t, t_))

100%|██████████| 1/1 [00:00<00:00, 2032.12it/s]


In [16]:
results

[([['2', 2], ['buah', 2], ['kereta', 2]],
  [['2', 2], ['angkatan', 10], ['kereta', 2]])]